In [83]:
import pandas as pd
import numpy as np
from matplotlib import style

from sklearn import preprocessing

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
from sklearn.neural_network import BernoulliRBM
from sklearn.mixture import BayesianGaussianMixture
from sklearn.manifold import Isomap

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.ensemble import AdaBoostClassifier
import random

In [84]:
#read in the data
df = pd.read_csv('diabetes.csv')

#columns to drop
#df = df.drop(['id'], axis=1)
df.sample(frac=1)

#gets rid of ? and one hot encoding for all columns that need it
index = []
count = 0
for val in range(len(df.ix[:,0])):
    flag = False
    for column in df:
        if df[column][val] == '?':
            flag = True
            break
    if flag:
        continue
    if count<1000:
        index.append(val)
        count += 1
df = df[df.index.isin(index)]

#gets all columns which are not ints and integer encodes them
obj_df = df.select_dtypes(include=['object']).copy()
for column in obj_df:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    df[column] = le.transform(df[column])
    
#normalize all points between [0,1]
x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [85]:
#make dataset only 1100
#create 500/500 split between labelled on nonlablled array, 1000 semi-sup data set, and 100 validation dataset
train, test = np.split(df.sample(frac=1), [int(.6*len(df))])
train = train.values.tolist()
test = test.values.tolist()

df_unsupervised = []

label_nolabels = {}
for point in train:
    #unlablled 1000 points data
    df_unsupervised.append(point[:-1])
    label_nolabels[tuple(point[:-1])]= point[-1:]

In [86]:
# #kmeans_forest 1-10, unsupervised learning adaboosting
# kmeans1 = KMeans(n_clusters=2).fit(df_unsupervised)
# # #kmeans2 = SpectralClustering(n_clusters = 2).fit_predict(df_unsupervised).tolist()
# # kmeans3 = MeanShift().fit(df_unsupervised)
# # #kmeans4 = AgglomerativeClustering(n_clusters=2).fit_predict(df_unsupervised).tolist()
# # kmeans5 = DBSCAN().fit_predict(df_unsupervised).tolist()
# # kmeans6 = GaussianMixture(n_components=2).fit(df_unsupervised)
# # kmeans7 = Birch(n_clusters=2).fit(df_unsupervised)
# # kmeans8 = BayesianGaussianMixture(n_components=2).fit(df_unsupervised)
# classifiers = [kmeans1, kmeans3, kmeans5, kmeans6, kmeans7, kmeans8]
#kmeans_forest 1-10, unsupervised learning adaboosting
kmeans1 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans2 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans3 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans4 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans5 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans6 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans7 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans8 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans9 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised))
kmeans10 = KMeans(n_clusters=2, init='random', n_init=10).fit(np.asarray(df_unsupervised)) 
classifiers = [kmeans1,kmeans2,kmeans3,kmeans4,kmeans5,kmeans6,kmeans7,kmeans8,kmeans9,kmeans10]

In [87]:
# make csv in form of rowNumber, clfNumber, clf prediction on that row
answers = []
for point in range(len(df_unsupervised)):
    for clf in range(len(classifiers)):
        answers.append([point, clf, classifiers[clf].predict([df_unsupervised[point]])])

count = 0
f = open("answer_file.csv", "w")
f.write('question,worker,answer;\n')
for answer in answers:
    count += 1
    f.write(str(answer[0]) + ',' + str(answer[1]) + ',' + str(int(answer[2]))+'\n')
f.close()
p = open("result_file.csv", "w")   
p.close()

In [88]:
#run VI BP
import subprocess
subprocess.call(["python", "run.py", "methods/c_EM/method.py", "answer_file.csv", "result_file.csv","decision-making"])

0

In [89]:
#extract results, get noisy labels and 
filepath = "result_file.csv"
noisy_labels = []
with open(filepath) as fp:  
    for line in fp:
        questionAnswer = line.split(',')
        noisy_labels.append(questionAnswer)

In [90]:
#assign noisy label to proper row
df_noise_x = [] 
df_noise_y = []
for question in noisy_labels:
    if question[0].rstrip() == 'question':
        continue
    df_noise_x += [df_unsupervised[int(question[0].rstrip())]]
    df_noise_y.append(int(question[1].rstrip()))
count_vi = 0
for el in range(len(df_noise_x)):
    if label_nolabels[tuple(df_noise_x[el])][0] != df_noise_y[el]:
        count_vi += 1
print(count_vi)

155


In [91]:
df_noise_y2 = []
for el in df_noise_y:
    df_noise_y2.append(int(el))

df_noise = []
for el in range(len(df_noise_x)):
    new = df_noise_x[el]
    new.append(df_noise_y2[el])
    df_noise.append(new)

#need to shuffle the data
random.shuffle(df_noise)

df_noise_x = []
df_noise_y = []
for row in df_noise:
    df_noise_x.append(row[:-1])
    df_noise_y.append(row[-1:][0])

In [92]:
#run AdaBoost from Sklearn on noisy data
bdt2 = AdaBoostClassifier(DecisionTreeClassifier(),
                         algorithm="SAMME",
                         n_estimators=200)
bdt2.fit(df_noise_x, df_noise_y)


AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=None)

In [93]:
#Ada boosting on noisy data error rate
errors = []
count1 = 0
for point in test:
    est = bdt2.predict([point[:-1]])
    true = int(point[-1:][0])
    est = int(est[0])
    if est == true:
        errors.append([point[:-1],0])
    else:
        count1 += 1
        errors.append([point[:-1],1])

#error rate, noisy -> baseline 
print(count1, count1/len(test))

106 0.34415584415584416
